<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/RAG/Knowledge_Graph/2_Extract_themes_Microsoft_Phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phi3
https://github.com/microsoft/Phi-3CookBook

In [1]:
!python -m pip install langchain langchain-google-genai langchain langchain-community langchain-experimental --quiet
! python -m pip install graphdatascience nltk  accelerate flash_attn einops timm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 84.4 MB/s eta 0:00:00
  Preparing metadata

In [2]:
! pip install -U accelerate transformers  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 52.2 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from graphdatascience import GraphDataScience
from getpass import getpass
import time
import numpy as np
import nltk
from google.oauth2 import service_account

import json

import os

# Set up connection information

In [5]:
neo4j_password = "impulse-quarter-plug"

In [6]:
neo4j_database_name = "neo4j"
neo4j_user = "neo4j"
bolt_url = "bolt://18.209.168.76:7687"
gds = GraphDataScience(bolt_url, auth=(neo4j_user, neo4j_password))

We use a Phi3 mini 4 k for cost reason, as we need to do multiple call (more than 10k to the interface and obviusly

In [8]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

messages = [
    {"role": "system", "content": "Your are a python developer."},
    {"role": "user", "content": "Help me generate a bubble algorithm"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 600,
    "return_full_text": False,
    "temperature": 0.3,
    "do_sample": True,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 Sure, I can help you with that. The bubble sort algorithm is a simple sorting algorithm that repeatedly steps through the list, compares adjacent elements and swaps them if they are in the wrong order. Here's a Python implementation of the bubble sort algorithm:

```python
def bubble_sort(arr):
    n = len(arr)
    
    # Traverse through all array elements
    for i in range(n):
        
        # Last i elements are already in place
        for j in range(0, n-i-1):
            
            # Traverse the array from 0 to n-i-1
            # Swap if the element found is greater than the next element
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]

# Example usage
arr = [64, 34, 25, 12, 22, 11, 90]
bubble_sort(arr)
print("Sorted array is:")
for i in range(len(arr)):
    print("%d" % arr[i])
```

This code defines a function `bubble_sort` that takes an array as input and sorts it using the bubble sort algorithm. The function uses two nested loops t

In [9]:
from langchain.llms.base import LLM

class CustomLLM(LLM):
    def _call(self, prompt, stop=None, run_manager=None) -> str:
        inputs = tokenizer(prompt, return_tensors="pt")
        inputs = inputs.to("cuda")
        result = model.generate(input_ids=inputs.input_ids, max_new_tokens=1024,  do_sample=False)
        #
        prompt_length = len(tokenizer.encode(prompt, return_tensors="pt")[0])
        generated_response = tokenizer.decode(
            result[0][prompt_length:], skip_special_tokens=True
        )
        # result = tokenizer.decode(result[0])
        return generated_response

    @property
    def _llm_type(self) -> str:
        return "custom"

llm = CustomLLM()

In [10]:
llm

CustomLLM()

# Get movie data

In [11]:
movie_info = gds.run_cypher(
    """MATCH (m:Movie)
       WHERE NOT EXISTS {(m)-[:HAS_THEME]->()}
       AND trim(m.overview) <> ''
       RETURN m.tmdbId AS tmdbId, m{.title, .overview} AS description""" )

/usr/local/lib/python3.10/dist-packages/graphdatascience/query_runner/neo4j_query_runner.py:224: RuntimeWarning: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: HAS_THEME)
  warnings.warn(warning)


In [12]:
movie_info.head()

,tmdbId,description
0,42050,"{'title': 'Seduction: The Cruel Woman', 'overv..."
1,65456,"{'title': 'The Atrocity Exhibition', 'overview..."
2,68621,"{'title': 'Serious Mixing (Part 1)', 'overview..."
3,187716,"{'title': 'Universal Groove', 'overview': 'Fil..."
4,269579,"{'title': 'A Circus Tale & A Love Song', 'over..."


In [13]:
len(movie_info)

25913

In [14]:
gds.run_cypher(
    """MATCH (m:Movie)
       WHERE NOT EXISTS {(m)-[:HAS_THEME]->()}
       AND trim(m.overview) = ''
       RETURN m.tmdbId AS tmdbId, m{.title, .overview} AS description""" )

/usr/local/lib/python3.10/dist-packages/graphdatascience/query_runner/neo4j_query_runner.py:224: RuntimeWarning: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: HAS_THEME)
  warnings.warn(warning)


,tmdbId,description
0,250316,"{'title': 'Der NSU-Prozess als Film', 'overvie..."
1,453035,"{'title': 'Um Filme de Cinema', 'overview': ''}"
2,552177,"{'title': 'Désirs de femmes', 'overview': ''}"
3,709304,"{'title': 'Korzenie zła', 'overview': ''}"
4,727342,"{'title': 'From the New World', 'overview': ''}"
...,...,...
5533,1313624,{'title': 'Brancusi : les métamorphoses de la ...
5534,1313659,"{'title': 'Horizon 25 & Béni', 'overview': ''}"
5535,1313701,"{'title': 'Querbeat: Southside Festival 2024',..."
5536,1313781,"{'title': 'Listras No Abismo', 'overview': ''}"


In [15]:
gds.run_cypher(
    """MATCH (m:Movie)
       WHERE NOT EXISTS {(m)-[:HAS_THEME]->()}
       AND trim(m.overview) = ''
       RETURN count(*)""" )

/usr/local/lib/python3.10/dist-packages/graphdatascience/query_runner/neo4j_query_runner.py:224: RuntimeWarning: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: HAS_THEME)
  warnings.warn(warning)


,count(*)
0,5538


In [18]:
gds.run_cypher(
    """match (m:Movie) return count(*)
""" )

,count(*)
0,31451


In [19]:
gds.run_cypher("""
MATCH (m:Movie)
       WHERE NOT EXISTS {(m)-[:HAS_THEME]->()}
       AND trim(m.overview) = ''
DETACH DELETE m""")

""


In [20]:
gds.run_cypher(
    """match (m:Movie) return count(*)
""" )

,count(*)
0,25913


In [21]:
movie_info[:1].to_dict()

{'tmdbId': {0: 42050},
 'description': {0: {'title': 'Seduction: The Cruel Woman',
   'overview': 'Wanda is a dominatrix who runs a gallery in a building on the Hamburg waterfront, where audiences pay for the privilege of watching her humiliate her slaves. She is a business woman who smashes sexual stereotypes and social taboos with icy self-possession and an enigmatic smile. As artist she specializes in the staging of elaborate BDSM fantasies and her affairs transgress the usual boundaries of personal and professional life. Along the way she leaves her German lesbian lover, a shoe fetishist, for an American "trainee," and does more than step on the toes of the male performer who has broken the rules of the master-slave relationship by falling in love with her.'}}}

# Create theme extraction pipeline

In [22]:
prompt_1 ="""You are a movie expert.
    You are given the tile and overview of the plot of a movie.
    Summarize the most memorable themes, settings, and public figures in the movie into a list of up to eight one-to-two word phrases.
    Only include the names of people if the person is a famous public figure.
    Prioritize any phrases that appear in the movie's title.
    You can provide fewer than eight phrases.
    very important:
    1) Return the phrases as a pipe separated list.
    2) Return only the list without a heading.
    title: {title}
    overview: {overview}
    """
chat_template = '<|user|>\n{input} <|end|>\n<|assistant|>'

template = f'{chat_template.format(input=prompt_1)}'

final_prompt = ChatPromptTemplate.from_template(template)



In [23]:
final_prompt

ChatPromptTemplate(input_variables=['overview', 'title'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['overview', 'title'], template="<|user|>\nYou are a movie expert. \n    You are given the tile and overview of the plot of a movie.\n    Summarize the most memorable themes, settings, and public figures in the movie into a list of up to eight one-to-two word phrases. \n    Only include the names of people if the person is a famous public figure.\n    Prioritize any phrases that appear in the movie's title.\n    You can provide fewer than eight phrases.\n    very important:\n    1) Return the phrases as a pipe separated list. \n    2) Return only the list without a heading.\n    title: {title}\n    overview: {overview} \n     <|end|>\n<|assistant|>"))])

In [24]:
summary_chain = final_prompt | llm

In [25]:
movie_info.loc[1, 'description']

{'title': 'The Atrocity Exhibition',
 'overview': 'A doctor in a mental research institution is driven insane by the spectacle of the horrors of the twentieth century.'}

In [26]:
out = summary_chain.invoke(movie_info.loc[1, 'description'])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [27]:
out

'Atrocity Exhibition | Doctor | Mental Research Institution | Twentieth Century | Insanity | Horrors | Spectacle | Public Figures'

In [28]:
test_batch = movie_info.loc[0:5,:]['description'].to_list()

In [29]:
summary_chain.batch(test_batch)

['Seduction | Cruel Woman | BDSM | Waterfront | Dominatrix | Wanda | German | American | Master-Slave',
 'Atrocity Exhibition | Doctor | Mental Research Institution | Twentieth Century | Insanity | Horrors | Spectacle | Public Figures',
 'Legend | DJ | Serious | Alan Walker | Illenium | Dj Khaled | Beat Matching | Nightclub | Studio',
 'Groove | Rave Culture | 90s | Escape | Self-Discovery | Drugs | Dance | Nostalgia | Underground Party',
 'Circus, Love, Boy, Leave, Life, Love Song, Public Figure, Memorable',
 'Beverly Hills | Axel Foley | Chaos | Detroit | Crime | Unforgettable | First Case | Forty Years']

# Extract themes and send to Neo4j
Create a node key constraint for the Theme nodes that requires that all Theme nodes have a unique value for the description property.

In [30]:
gds.run_cypher("""CREATE CONSTRAINT theme_node_key IF NOT EXISTS FOR (n:Theme) REQUIRE n.description IS NODE KEY""")

""


In [31]:
def process_theme_batch(start_index, end_index):
    rows = movie_info.iloc[start_index: end_index,:].copy()
    results = summary_chain.batch(rows['description'].to_list())
    # r= [result.content.split("|") for result in results]
    themes =[]
    for result in results:
        if result != "":
            themes.append(result.replace(",", "|").split("|"))
        else:
            themes.append([])
    rows['themes'] = themes
    gds.run_cypher("""
    UNWIND $data as d
    MATCH (m:Movie {tmdbId:d['tmdbId']})
    UNWIND d['themes'] AS theme
    MERGE (t:Theme {description:trim(theme)})
    WITH t, m
    MERGE (m)-[:HAS_THEME {prompt: "v2"}]->(t)
    """,
                   {"data": rows[['tmdbId', 'themes']].to_dict("records")})

In [32]:
for i in range(0, int(movie_info.shape[0]/10) + 1):
    process_theme_batch(i*10,(i+1)*10)
    if (i + 1) % 10 == 0:
        print(f"Finished row {(i+1)*10}")
    time.sleep(1)

Finished row 100
Finished row 200
Finished row 300
Finished row 400
Finished row 500
Finished row 600
Finished row 700
Finished row 800
Finished row 900
Finished row 1000
Finished row 1100
Finished row 1200
Finished row 1300
Finished row 1400
Finished row 1500
Finished row 1600
Finished row 1700
Finished row 1800
Finished row 1900
Finished row 2000
Finished row 2100
Finished row 2200
Finished row 2300
Finished row 2400
Finished row 2500
Finished row 2600
Finished row 2700
Finished row 2800
Finished row 2900
Finished row 3000
Finished row 3100
Finished row 3200
Finished row 3300
Finished row 3400
Finished row 3500
Finished row 3600
Finished row 3700
Finished row 3800
Finished row 3900
Finished row 4000
Finished row 4100
Finished row 4200
Finished row 4300
Finished row 4400
Finished row 4500
Finished row 4600
Finished row 4700
Finished row 4800
Finished row 4900
Finished row 5000
Finished row 5100
Finished row 5200
Finished row 5300
Finished row 5400
Finished row 5500
Finished row 5600
F